In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

In [ ]:
filename_grokKey = "..\\GroqAPIKey.txt"

In [6]:
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### Runnable with Message History with session ID

In [11]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory,ChatMessageHistory


In [8]:

model = ChatGroq(model="llama-3.1-8b-instant",temperature=0)


In [12]:
prompt = ChatPromptTemplate(
    [
        (
            "system","You are having a chat with human."
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human","{input}")
    ]
)

In [13]:
chain = prompt | model | StrOutputParser()

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [15]:
store = {}

In [16]:
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    
    return store[session_id]

In [17]:
runnable = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [18]:
runnable.invoke(
    {"input":"Which country is the biggest exporter of cotton?"},
    config={"configurable":{"session_id":"sess1"}},
)

"China is the world's largest producer of cotton, but when it comes to exports, the largest exporter of cotton is actually the United States."

In [19]:
#Remembers
runnable.invoke(
    {"input":"What is the total value of cotton exports of this country?"},
    config={"configurable":{"session_id":"sess1"}},
)

'The total value of cotton exports of the United States can vary from year to year, depending on factors such as global demand, production levels, and market prices.\n\nHowever, according to the United States Department of Agriculture (USDA), in the 2020/21 marketing year, the total value of U.S. cotton exports was around $6.4 billion.'

In [20]:
#New session id --> does not remember
runnable.invoke(
    {"input":"What is the total value of cotton exports of this country?"},
    config={"configurable":{"session_id":"sess2"}},
)

"I'd be happy to help you with that. However, I need more information about the country you're referring to. Could you please tell me which country you're interested in knowing the total value of cotton exports for?"